# MusicBrainz artist lookup

Documentation:
 - Web service: https://wiki.musicbrainz.org/Development/XML_Web_Service/Version_2/Search
 - Artist entities: https://musicbrainz.org/doc/Artist
 - Area entities: https://musicbrainz.org/doc/Area

Sample queries:
 - Artist: https://musicbrainz.org/ws/2/artist/d4659efb-b8eb-4f03-95e9-f69ce35967a9
 - Area: https://musicbrainz.org/ws/2/area/0a70f24b-1263-4341-8d70-17b8df84154f?inc=area-rels

In [1]:
import pandas as pd, requests, time, json

pause_standard = 0.1
pause_exceeded_rate = 19

In [2]:
# configure URLs and user-agent header
artist_name_url = u'https://musicbrainz.org/ws/2/artist/?query=artist:{}&fmt=json'
artist_id_url = u'https://musicbrainz.org/ws/2/artist/{}?fmt=json'
area_id_url = u'https://musicbrainz.org/ws/2/area/{}?inc=area-rels&fmt=json'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [3]:
def make_request(url, headers=headers):
    
    global pause_standard
    global pause_exceeded_rate
    
    time.sleep(pause_standard)
    #print url
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200: #if status OK
        return {'status_code':response.status_code, 'json':response.json()}
    
    elif response.status_code == 503: #if status error (server busy or rate limit exceeded)
        if 'exceeding the allowable rate limit' in response.json()['error']:
            #print '\n{}\n{}\npausing {} secs and re-try\n'.format(response.status_code, response.json(), pause_exceeded_rate)
            pause_standard = pause_standard + 0.1
            print 'pause_standard={}'.format(pause_standard),
            time.sleep(pause_exceeded_rate)
        return make_request(url)
    
    else: #if other status code, print info and return None for caller to handle
        print '\nmake_request error code {} {}\n'.format(response.status_code, response.json())
        return None

In [4]:
def geocode(address):
    time.sleep(pause) #pause for some duration before each request, to not hammer their server
    url = u'http://maps.googleapis.com/maps/api/geocode/json?address={}&sensor=false' #api url with placeholders
    request = url.format(address) #fill in the placeholder with a variable
    response = requests.get(request) #send the request to the server and get the response
    data = json.loads(response.text) #convert the response json string into a dict
    
    if len(data['results']) > 0: #if google was able to geolocate our address, extract lat-long from result
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return '{},{}'.format(latitude, longitude)

In [5]:
def get_artist_id_by_name(name):
    try:
        response = make_request(artist_name_url.format(name))
        if response is not None:
            result = response['json']
            artist_id = result['artists'][0]['id']
            return artist_id
    except:
        print 'get_artist_id_by_name error: {}'.format(response)

In [6]:
def get_artist_by_id(artist_id):
    try:
        response = make_request(artist_id_url.format(artist_id))
        if response is not None:
            result = response['json']
            data = {'id':artist_id,
                    'name':result['name'],            
                    'type':result['type'],
                    'gender':result['gender'],
                    'country':result['country'],
                    'begin_date':None,
                    'end_date':None,
                    'area_id':None,
                    'area_name':None,
                    'area_name_full':None,
                    'area_latlng':None,
                    'begin_area_id':None,
                    'begin_area_name':None,
                    'begin_area_name_full':None,
                    'begin_area_latlng':None,
                    'place':None,
                    'place_id':None,
                    'place_full':None,
                    'place_latlng':None}

            if result['life-span'] is not None and 'begin' in result['life-span'] and 'end' in result['life-span']:
                data['begin_date'] = result['life-span']['begin']
                data['end_date'] = result['life-span']['end']
            if result['area'] is not None and 'id' in result['area'] and 'name' in result['area']:
                data['area_id'] = result['area']['id']
                data['area_name'] = result['area']['name']
            if result['begin_area'] is not None and 'id' in result['begin_area'] and 'name' in result['begin_area']:
                data['begin_area_id'] = result['begin_area']['id']
                data['begin_area_name'] = result['begin_area']['name']
            
            # populate place with begin_area_name if it's not null, else area_name if it's not null, else None
            if data['begin_area_name'] is not None:
                data['place'] = data['begin_area_name']
                data['place_id'] = data['begin_area_id']
            elif data['area_name'] is not None:
                data['place'] = data['area_name']
                data['place_id'] = data['area_id']
            
            return data
    
    except:
        print 'get_artist_by_id error: {}'.format(response)

In [7]:
def get_area(area_id, area_str=''):
    try: 
        response = make_request(area_id_url.format(area_id))
        if response is not None:
            result = response['json']
            
            if area_str == '':
                area_str = result['name']
            
            if 'relations' in result:
                for relation in result['relations']:
                    if relation['direction']=='backward' and relation['type']=='part of':
                        area_id = relation['area']['id']
                        area_str = u'{}, {}'.format(area_str, relation['area']['name'])
                        return area_id, area_str
            else:
                print 'get_area no relations error: {}'.format(result)
            return None, area_str
    except:
        print 'get_area error: {}'.format(response)

In [8]:
def get_place_full_name_by_id(area_id):
    area_name=''
    while area_id is not None:
        area_id, area_name = get_area(area_id, area_name)
    return area_name

## Test it with a demo

In [9]:
name = 'david bowie'
artist_id = get_artist_id_by_name(name)
artist = get_artist_by_id(artist_id)
artist['place_full'] = get_place_full_name_by_id(artist['place_id'])
artist['place_full']

u'Brixton, Lambeth, London, England, United Kingdom'

## Now run it

In [10]:
df = pd.DataFrame(columns=['id', 'name', 'type', 'gender', 'country', 'begin_date', 'end_date', 
                           'begin_area_name', 'begin_area_name_full', 'begin_area_latlng', 
                           'area_name', 'area_name_full', 'area_latlng',
                           'place_id', 'place', 'place_full', 'place_latlng'])

In [11]:
scrobbles = pd.read_csv('data/lastfm_scrobbles.csv', encoding='utf-8')
artist_ids = scrobbles['artist_mbid'].dropna().unique()
len(artist_ids)

12484

In [12]:
start_time = time.time()

for artist_id, n in zip(artist_ids, range(len(artist_ids))):
    
    if n % 10 == 0 :
        print n,
        df.to_csv('data/mb.csv', index=False, encoding='utf-8')
        
    try:
        # get the artist info object
        artist = get_artist_by_id(artist_id)
        
        # create a new df row containing the data from this artist object
        df.loc[n] = [artist['id'], artist['name'], artist['type'], artist['gender'], artist['country'], 
                     artist['begin_date'], artist['end_date'],
                     artist['begin_area_name'], artist['begin_area_name_full'], artist['begin_area_latlng'],
                     artist['area_name'], artist['area_name_full'], artist['area_latlng'],
                     artist['place_id'], artist['place'], artist['place_full'], artist['place_latlng']]
        
        # update the row's place_full cell with the full place name
        df.loc[n, 'place_full'] = get_place_full_name_by_id(artist['place_id'])
        
    except:
        print '#{} failed'.format(n)
        pass

df.to_csv('data/mb.csv', index=False, encoding='utf-8')
finish_time = time.time()

0 10 pause_standard=0.2 20 30 pause_standard=0.3 40 pause_standard=0.4 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 pause_standard=0.5 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 
make_request error code 404 {u'error': u'Not Found'}

#1286 failed
1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 19

In [13]:
print 'processed {:,} artists in {:,} seconds'.format(len(artist_ids), round(finish_time-start_time, 2))
df

processed 12,484 artists in 67,551.32 seconds


,id,name,type,gender,country,begin_date,end_date,begin_area_name,begin_area_name_full,begin_area_latlng,area_name,area_name_full,area_latlng,place_id,place,place_full,place_latlng
0,6c85f4c3-026c-4c16-9a7c-f546f42ed0fb,Guided by Voices,Group,None,US,1983,2014-09,Dayton,None,None,United States,None,None,a55188f6-e976-499b-97b2-735985458216,Dayton,"Dayton, Montgomery County, Ohio, United States",None
1,ad82dd72-0df3-4a09-8d7a-1af9c9e80522,Patsy Cline,Person,Female,US,1932-09-08,1963-03-05,Winchester,None,None,United States,None,None,17160c4f-5390-453c-a901-0bb5529f186e,Winchester,"Winchester, Virginia, United States",None
2,98f42ac3-7892-475e-ab47-0a55a3478b18,Radiator Hospital,Group,None,None,None,None,None,None,None,None,None,None,None,None,,None
3,0110e63e-0a9b-4818-af8e-41e180c20b9a,Devendra Banhart,Person,Male,US,1981-05-30,None,Houston,None,None,United States,None,None,c920948b-83e3-40b7-8fe9-9ab5abaac55b,Houston,"Houston, Harris County, Texas, United States",None
4,fa690c23-512b-4cad-aa32-c318f4228947,Nico,Person,Female,RO,1970-02-01,None,None,None,None,Romania,None,None,61ed84b8-5a10-30a7-8376-ccd51801d6d1,Romania,Romania,None
5,ea075268-e5ea-40f7-b9c3-68b039218a88,The Apples in Stereo,Group,None,US,1993,None,Denver,None,None,United States,None,None,fc1aee9a-f1a8-45dc-8820-af6b5d7f7450,Denver,"Denver, Colorado, United States",None
6,99524243-dc2e-4bf3-82c0-72395391d21f,Scott Walker,Person,Male,US,1943-01-09,None,None,None,None,United States,None,None,489ce91b-6658-3307-9877-795b68554c98,United States,United States,None
7,3ff72a59-f39d-411d-9f93-2d4a86413013,The Magnetic Fields,Group,None,US,1990,None,Boston,None,None,United States,None,None,e331bfdf-b908-429c-a79b-710cf9c06abb,Boston,"Boston, Suffolk County, Massachusetts, United ...",None
8,3533dea4-218c-4fc9-ab6e-2e3dc0ffde6e,Pete Yorn,Person,Male,US,1974-07-27,None,None,None,None,United States,None,None,489ce91b-6658-3307-9877-795b68554c98,United States,United States,None
9,a41ac10f-0a56-4672-9161-b83f9b223559,Van Morrison,Person,Male,None,1945-08-31,None,Belfast,None,None,Northern Ireland,None,None,057ce294-2684-46fd-aee1-4d8c6b4bf4ef,Belfast,"Belfast, County Down, Northern Ireland, Ireland",None
